In [3]:


get_ipython().system(' pip install torch transformers sentencepiece')

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvjitlink_cu

In [4]:


get_ipython().system(' pip install regex protobuf')

In [5]:


get_ipython().system(' pip install Bio')

In [7]:


import torch

In [8]:


from transformers import T5Tokenizer, T5EncoderModel
from Bio import SeqIO
import torch
import pandas as pd
import re
import os


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/disk2/arfa/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/disk2/arfa/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/disk2/arfa/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/disk2/arfa/an

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



RuntimeError: Failed to import transformers.models.t5.modeling_t5 because of the following error (look up to see its traceback):
numpy.core.multiarray failed to import

In [ ]:


# Clear CUDA cache before loading the model
torch.cuda.empty_cache()

In [ ]:


# Define device
device = torch.device('cuda')

In [ ]:


# Load ProtT5 model and tokenizer
transformer_link = "Rostlab/prot_t5_xl_uniref50"
print("Loading: {}".format(transformer_link))
model = T5EncoderModel.from_pretrained(transformer_link).to(device)
model.eval()  # Set model to evaluation mode

In [ ]:


tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False, legacy=True)

In [ ]:


# Function to process sequences and generate embeddings
def generate_embeddings(fasta_file, label):
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Replace rare/ambiguous amino acids by X
        sequence = re.sub(r"[UZOB]", "X", str(record.seq))
        sequences.append(sequence)

    embeddings = []
    for sequence in sequences:
        # Tokenize and pad sequences
        sequence_with_spaces = " ".join(list(sequence))
        ids = tokenizer.encode_plus(sequence_with_spaces, add_special_tokens=True, padding="longest", return_tensors="pt")

        input_ids = ids['input_ids'].to(device)
        attention_mask = ids['attention_mask'].to(device)

        # Generate embeddings
        with torch.no_grad():
            embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

        # Extract per-protein embedding
        per_protein_embedding = embedding_repr.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        embeddings.append(per_protein_embedding)

    # Create a DataFrame with embeddings and labels
    df_embeddings = pd.DataFrame(embeddings)
    df_embeddings['label'] = label  # Assign label based on input

    return df_embeddings

In [ ]:


# File paths for samples

file_paths = {
    'neg_train' : 'Dataset/S_tr_nega.fa',
    'pos_train' : 'Dataset/S_tr_posi.fa',
    'neg_test' : 'Dataset/S_tst_nega.fa',
    'pos_test' : 'Dataset/S_tst_posi.fa'
}

In [ ]:


# Generate embeddings and save each DataFrame separately
for file_type, path in file_paths.items():
    if "neg" in file_type:
        label = 1  # Label for negative samples
    else:
        label = 0  # Label for positive samples

    embeddings_df = generate_embeddings(path, label)

    # Save each embeddings DataFrame to a separate CSV file
    output_file = f"{file_type}_embeddings.csv"
    embeddings_df.to_csv(output_file, index=False)
    print(f"Embeddings saved to {output_file}")

In [ ]:


#fasta_file = 'Dataset/S_tr_nega.fa'


# sequences = []
# for record in SeqIO.parse(fasta_file, "fasta"):
#         # Replace rare/ambiguous amino acids by X
#     sequence = re.sub(r"[UZOB]", "X", str(record.seq))
#     sequences.append(sequence)

In [ ]:


### Train the classifier for DNA binding proteins

In [ ]:


##train
df1=pd.read_csv("neg_train_embeddings.csv")
df2=pd.read_csv("pos_train_embeddings.csv")
###test
df3=pd.read_csv("neg_test_embeddings.csv")
df4=pd.read_csv("pos_test_embeddings.csv")

df1.shape,df2.shape,df3.shape,df4.shape

In [ ]:


# Step 1: Concatenate the DataFrames
combined_df_train = pd.concat([df1, df2], ignore_index=True)

# Step 2: Shuffle the combined DataFrame
df_train = combined_df_train.sample(frac=1).reset_index(drop=True)

In [ ]:


#Step 1: Concatenate the DataFrames
combined_df_test = pd.concat([df3, df4], ignore_index=True)

# Step 2: Shuffle the combined DataFrame
df_test = combined_df_test.sample(frac=1).reset_index(drop=True)

In [ ]:


#### classifier

In [ ]:


get_ipython().system(' pip install scikit-learn matplotlib')

In [ ]:


import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt




X_train= df_train.iloc[:,:-1]
y_train= df_train.iloc[:,-1]
X_test= df_test.iloc[:,:-1]
y_test= df_test.iloc[:,-1]

# Step 3: Initialize and train the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = mlp.predict(X_test)
y_pred_proba = mlp.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Step 5: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Step 6: Calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Step 7: Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()